<h1>Mean Reversion Trading Algorithm </h1>

<ul>
    <li> Trading strategy that focuses on when a security moves away from the average. </li>
    <li> General idea is that it will move back toward the mean (at some point). </li>
    <li> A few ways to look at this strategy are, linear regression, moving average. </li>
    <li> How far of the movement is too far? </li>
</ul>

In [2]:
import pandas as pd
import pandas_datareader as pdr
import numpy as np
import yfinance as fin
import datetime as dt
import quandl as qdl
import os
from scipy import stats
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams['figure.figsize'] = 8,6
import seaborn as sb
sb.set()

<h5> Mean Reversion strategy assumes that a security is going to move back to the some kind of average whenever it moves too far away. Basically, it means it would move back to the mean at some point.</h5>

In [16]:
start_date = dt.datetime(2012,5,18)
end_date = dt.datetime(2023,1,10)
gold = fin.Ticker('GLD')
gold_hist = pd.DataFrame(gold.history(start = start_date, end = end_date)['Close'])

In [17]:
gold_hist.head()

,Close
Date,
2012-05-18 00:00:00-04:00,154.550003
2012-05-21 00:00:00-04:00,154.649994
2012-05-22 00:00:00-04:00,152.080002
2012-05-23 00:00:00-04:00,151.619995
2012-05-24 00:00:00-04:00,151.410004


In [21]:
moving_avg = 21
gold_hist['Returns'] = np.exp(gold_hist['Close']).diff()
gold_hist['Moving Avg'] = gold_hist['Close'].rolling(moving_avg).mean()

# The ratio will decide if we go long or short
## If we are far above it we would go short and we are too far below we will go long
gold_hist['Ratio'] = gold_hist['Close']/gold_hist['Moving Avg']

In [20]:
gold_hist['Ratio'].describe()

count    2658.000000
mean        1.000453
std         0.023308
min         0.862830
25%         0.986657
50%         1.000400
75%         1.015304
max         1.098813
Name: Ratio, dtype: float64

In [22]:
percentile = [5, 10, 50, 90, 95]
np.percentile(gold_hist['Ratio'].dropna(), percentile)

array([0.96309618, 0.97200687, 1.00040039, 1.02846414, 1.03704418])